# Qwen-Image-Edit-2509 LoRA Training

Notebook để train LoRA cho Qwen-Image-Edit-2509 trên RunPod.

## Checklist trước khi chạy:
- [ ] Đã upload dataset vào thư mục `dataset/gendata/vietnamese_dataset_qwen_edit/`
- [ ] Đã tải models vào thư mục `models/` (tải trực tiếp, không cần giữ cấu trúc thư mục gốc):
  - **DiT Edit-2509**: `models/qwen_image_edit_2509_bf16.safetensors`
    - Tải từ: https://huggingface.co/Comfy-Org/Qwen-Image-Edit_ComfyUI
    - File gốc: `split_files/diffusion_models/qwen_image_edit_2509_bf16.safetensors`
  - **VAE**: `models/diffusion_pytorch_model.safetensors`
    - Tải từ: https://huggingface.co/Qwen/Qwen-Image
    - File gốc: `vae/diffusion_pytorch_model.safetensors`
  - **Text Encoder**: `models/qwen_2.5_vl_7b.safetensors`
    - Tải từ: https://huggingface.co/Comfy-Org/Qwen-Image_ComfyUI
    - File gốc: `split_files/text_encoders/qwen_2.5_vl_7b.safetensors`
- [ ] Đã cài đặt dependencies (chạy cell đầu tiên)


## 0. Install Dependencies (Chạy cell này TRƯỚC TIÊN)


In [ ]:
# Install PyTorch và dependencies
# QUAN TRỌNG: Chạy cell này TRƯỚC TIÊN trước khi chạy các cell khác

import subprocess
import sys

print("Installing dependencies...")
print("=" * 60)

# Install PyTorch với CUDA (điều chỉnh cu124 hoặc cu128 tùy CUDA version)
# RunPod thường dùng CUDA 12.4, nếu khác thì sửa cu124 thành cu128
print("Step 1: Installing PyTorch with CUDA...")
result = subprocess.run([
    sys.executable, "-m", "pip", "install", 
    "torch", "torchvision", 
    "--index-url", "https://download.pytorch.org/whl/cu124"
], check=False)

if result.returncode != 0:
    print("⚠ Warning: PyTorch installation may have failed. Trying cu128...")
    result = subprocess.run([
        sys.executable, "-m", "pip", "install", 
        "torch", "torchvision", 
        "--index-url", "https://download.pytorch.org/whl/cu128"
    ], check=False)

# Install project dependencies
print("\nStep 2: Installing project dependencies...")
result = subprocess.run([
    sys.executable, "-m", "pip", "install", "-e", "."
], check=False)

if result.returncode == 0:
    print("\n" + "=" * 60)
    print("✓ Dependencies installed successfully!")
    print("=" * 60)
else:
    print("\n" + "=" * 60)
    print("❌ Dependencies installation failed!")
    print("Please check the error messages above.")
    print("=" * 60)


## 1. Cấu hình Paths và Parameters


In [ ]:
# ============================================
# CONFIGURATION - Điều chỉnh các paths này
# ============================================

# Base directory (thư mục gốc của repo)
BASE_DIR = "."

# Model paths
DIT_MODEL = "models/qwen_image_edit_2509_bf16.safetensors"
VAE_MODEL = "models/diffusion_pytorch_model.safetensors"
TEXT_ENCODER_MODEL = "models/qwen_2.5_vl_7b.safetensors"

# Dataset config
DATASET_CONFIG = "dataset/gendata/vietnamese_dataset_qwen_edit/dataset_config.toml"

# Output directory
OUTPUT_DIR = "output"
OUTPUT_NAME = "qwen_edit_2509_lora"

# Training parameters
LEARNING_RATE = 5e-5
MAX_TRAIN_EPOCHS = 16
SAVE_EVERY_N_EPOCHS = 1
NETWORK_DIM = 16
SEED = 42

# Memory optimization (cho RTX 3090 hoặc GPU tương tự)
USE_FP8_DIT = True  # --fp8_base --fp8_scaled
USE_FP8_VL = True   # --fp8_vl
USE_GRADIENT_CHECKPOINTING = True
USE_BLOCKS_TO_SWAP = False  # Set True nếu vẫn thiếu VRAM, cần 64GB RAM
BLOCKS_TO_SWAP = 16

print("✓ Configuration loaded")
print(f"Dataset config: {DATASET_CONFIG}")
print(f"Output dir: {OUTPUT_DIR}")
print(f"Output name: {OUTPUT_NAME}")


## 2. Kiểm tra Files và Dependencies


In [ ]:
import os
from pathlib import Path

# Kiểm tra các file cần thiết
def check_file(path, name):
    full_path = Path(BASE_DIR) / path
    if full_path.exists():
        print(f"✓ {name}: {full_path}")
        return True
    else:
        print(f"❌ {name} NOT FOUND: {full_path}")
        return False

print("Checking required files...")
print("=" * 60)

files_ok = True
files_ok &= check_file(DIT_MODEL, "DiT Model")
files_ok &= check_file(VAE_MODEL, "VAE Model")
files_ok &= check_file(TEXT_ENCODER_MODEL, "Text Encoder Model")
files_ok &= check_file(DATASET_CONFIG, "Dataset Config")

# Kiểm tra dataset directories
dataset_dir = Path(BASE_DIR) / "dataset/gendata/vietnamese_dataset_qwen_edit"
if (dataset_dir / "images").exists():
    image_count = len(list((dataset_dir / "images").glob("*.png")))
    print(f"✓ Images directory: {image_count} images found")
else:
    print(f"❌ Images directory NOT FOUND: {dataset_dir / 'images'}")
    files_ok = False

if (dataset_dir / "controls").exists():
    control_count = len(list((dataset_dir / "controls").glob("*.png")))
    print(f"✓ Controls directory: {control_count} control images found")
else:
    print(f"❌ Controls directory NOT FOUND: {dataset_dir / 'controls'}")
    files_ok = False

print("=" * 60)
if files_ok:
    print("✓ All files found! Ready to proceed.")
else:
    print("❌ Some files are missing. Please check the paths above.")


## 3. Cache Latents (Bước 1/3)


In [ ]:
# Cache latents cho Edit-2509
# QUAN TRỌNG: Phải dùng --edit_plus flag

import subprocess
import sys
import os
from pathlib import Path

# Đảm bảo đang ở đúng directory
os.chdir(BASE_DIR)

# Kiểm tra VAE model file trước khi chạy
vae_path = Path(BASE_DIR) / VAE_MODEL
print("Validating VAE model file...")
print("=" * 60)

if not vae_path.exists():
    print(f"❌ ERROR: VAE model file not found: {vae_path}")
    print(f"   Please check the path: {VAE_MODEL}")
    print(f"   Expected location: {vae_path.resolve()}")
    sys.exit(1)

# Kiểm tra file size (VAE model thường > 100MB)
file_size_mb = vae_path.stat().st_size / (1024 * 1024)
print(f"✓ VAE file found: {vae_path}")
print(f"  File size: {file_size_mb:.2f} MB")

if file_size_mb < 10:
    print(f"\n❌ ERROR: VAE file seems too small ({file_size_mb:.2f} MB)")
    print("   VAE model should be > 100MB. File may be corrupted or incomplete.")
    print("\n📥 CÁCH TẢI LẠI VAE MODEL:")
    print("   1. Xóa file hiện tại: rm models/diffusion_pytorch_model.safetensors")
    print("   2. Tải lại từ HuggingFace:")
    print("      - URL: https://huggingface.co/Qwen/Qwen-Image")
    print("      - File: vae/diffusion_pytorch_model.safetensors")
    print("      - Lưu vào: models/diffusion_pytorch_model.safetensors")
    print("   3. Đảm bảo file size > 100MB sau khi tải xong")
    sys.exit(1)
elif file_size_mb < 100:
    print(f"⚠ WARNING: VAE file seems small ({file_size_mb:.2f} MB)")
    print("   Expected size: > 100MB. File may be corrupted.")

# Kiểm tra file extension
if vae_path.suffix != '.safetensors':
    print(f"⚠ WARNING: VAE file extension is {vae_path.suffix}, expected .safetensors")

# Thử đọc header để kiểm tra file có hợp lệ không
try:
    import struct
    with open(vae_path, 'rb') as f:
        # Đọc magic number của safetensors (8 bytes)
        magic = f.read(8)
        if magic != b'__SF__\x00':
            print("⚠ WARNING: File may not be a valid safetensors file (magic number mismatch)")
        else:
            print("✓ File format check passed (safetensors magic number OK)")
except Exception as e:
    print(f"⚠ WARNING: Could not verify file format: {e}")

# Script path
script_path = "src/musubi_tuner/qwen_image_cache_latents.py"
if not Path(script_path).exists():
    script_path = "qwen_image_cache_latents.py"
    if not Path(script_path).exists():
        print(f"❌ ERROR: Script not found")
        sys.exit(1)

cmd = [
    sys.executable,
    script_path,
    "--dataset_config", DATASET_CONFIG,
    "--vae", VAE_MODEL,
    "--edit_plus"  # ← Flag cho Edit-2509 (theo docs)
]

print("\n" + "=" * 60)
print("Starting latent caching...")
print(f"Command: {' '.join(cmd)}")
print("=" * 60)

result = subprocess.run(cmd, cwd=BASE_DIR)

if result.returncode == 0:
    print("=" * 60)
    print("✓ Latent caching completed successfully!")
    print("=" * 60)
else:
    print("=" * 60)
    print("❌ Latent caching failed!")
    print("\n🔧 TROUBLESHOOTING:")
    print("\n1. VAE model file may be CORRUPTED:")
    print(f"   - File: {vae_path}")
    print(f"   - Size: {file_size_mb:.2f} MB (should be > 100MB)")
    print("   - Error: 'HeaderTooLarge' or 'Header size exceeds maximum'")
    print("\n   📥 CÁCH TẢI LẠI:")
    print("   a) Xóa file cũ:")
    print(f"      rm {vae_path}")
    print("   b) Tải lại từ HuggingFace:")
    print("      - URL: https://huggingface.co/Qwen/Qwen-Image")
    print("      - File: vae/diffusion_pytorch_model.safetensors")
    print("      - Lưu vào: models/diffusion_pytorch_model.safetensors")
    print("   c) Kiểm tra file size phải > 100MB")
    print("\n2. Check dataset config paths are correct (relative paths)")
    print("\n3. Check error messages above for more details")
    print("=" * 60)
    sys.exit(1)


## 4. Cache Text Encoder Outputs (Bước 2/3)


In [ ]:
# Cache text encoder outputs cho Edit-2509
# QUAN TRỌNG: Phải dùng --edit_plus flag

cmd = [
    sys.executable,
    "src/musubi_tuner/qwen_image_cache_text_encoder_outputs.py",
    "--dataset_config", DATASET_CONFIG,
    "--text_encoder", TEXT_ENCODER_MODEL,
    "--edit_plus",  # ← Flag cho Edit-2509
    "--batch_size", "1"
]

# Thêm --fp8_vl nếu cần tiết kiệm VRAM
if USE_FP8_VL:
    cmd.append("--fp8_vl")

print("Starting text encoder output caching...")
print(f"Command: {' '.join(cmd)}")
print("=" * 60)

result = subprocess.run(cmd, cwd=BASE_DIR)

if result.returncode == 0:
    print("=" * 60)
    print("✓ Text encoder output caching completed successfully!")
else:
    print("=" * 60)
    print("❌ Text encoder output caching failed!")
    sys.exit(1)


## 5. Train LoRA (Bước 3/3)


In [ ]:
# Train LoRA cho Edit-2509
# QUAN TRỌNG: Phải dùng --edit_plus flag

import os

# Tạo output directory nếu chưa có
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Build command
cmd = [
    "accelerate", "launch",
    "--num_cpu_threads_per_process", "1",
    "--mixed_precision", "bf16",
    "src/musubi_tuner/qwen_image_train_network.py",
    "--dit", DIT_MODEL,
    "--vae", VAE_MODEL,
    "--text_encoder", TEXT_ENCODER_MODEL,
    "--dataset_config", DATASET_CONFIG,
    "--edit_plus",  # ← Flag cho Edit-2509 (BẮT BUỘC)
    "--sdpa",
    "--mixed_precision", "bf16",
    "--timestep_sampling", "shift",
    "--weighting_scheme", "none",
    "--discrete_flow_shift", "2.2",
    "--optimizer_type", "adamw8bit",
    "--learning_rate", str(LEARNING_RATE),
    "--max_data_loader_n_workers", "2",
    "--persistent_data_loader_workers",
    "--network_module", "networks.lora_qwen_image",
    "--network_dim", str(NETWORK_DIM),
    "--max_train_epochs", str(MAX_TRAIN_EPOCHS),
    "--save_every_n_epochs", str(SAVE_EVERY_N_EPOCHS),
    "--seed", str(SEED),
    "--output_dir", OUTPUT_DIR,
    "--output_name", OUTPUT_NAME
]

# Memory optimization flags
if USE_FP8_DIT:
    cmd.extend(["--fp8_base", "--fp8_scaled"])
    print("✓ Using FP8 optimization for DiT (--fp8_base --fp8_scaled)")

if USE_FP8_VL:
    cmd.append("--fp8_vl")
    print("✓ Using FP8 optimization for Text Encoder (--fp8_vl)")

if USE_GRADIENT_CHECKPOINTING:
    cmd.append("--gradient_checkpointing")
    print("✓ Using gradient checkpointing")

if USE_BLOCKS_TO_SWAP:
    cmd.extend(["--blocks_to_swap", str(BLOCKS_TO_SWAP)])
    print(f"✓ Using block swapping (--blocks_to_swap {BLOCKS_TO_SWAP})")

print("\n" + "=" * 60)
print("Starting LoRA training for Edit-2509...")
print("=" * 60)
print(f"Full command:")
print(" ".join(cmd))
print("=" * 60 + "\n")

# Chạy training
result = subprocess.run(cmd, cwd=BASE_DIR)

if result.returncode == 0:
    print("\n" + "=" * 60)
    print("✓ Training completed successfully!")
    print(f"✓ LoRA saved to: {OUTPUT_DIR}/{OUTPUT_NAME}.safetensors")
    print("=" * 60)
else:
    print("\n" + "=" * 60)
    print("❌ Training failed!")
    print("=" * 60)
    sys.exit(1)


In [ ]:
# Kiểm tra các file output đã được tạo
from pathlib import Path

output_path = Path(OUTPUT_DIR)

print(f"Checking output directory: {output_path}")
print("=" * 60)

if output_path.exists():
    # Tìm các file LoRA
    lora_files = list(output_path.glob(f"{OUTPUT_NAME}*.safetensors"))
    
    if lora_files:
        print(f"✓ Found {len(lora_files)} LoRA file(s):")
        for f in sorted(lora_files):
            size_mb = f.stat().st_size / (1024 * 1024)
            print(f"  - {f.name} ({size_mb:.2f} MB)")
    else:
        print(f"⚠ No LoRA files found matching '{OUTPUT_NAME}*.safetensors'")
    
    # Liệt kê tất cả files
    all_files = list(output_path.glob("*"))
    if all_files:
        print(f"\nAll files in output directory ({len(all_files)}):")
        for f in sorted(all_files)[:20]:  # Show first 20
            if f.is_file():
                size_mb = f.stat().st_size / (1024 * 1024)
                print(f"  - {f.name} ({size_mb:.2f} MB)")
            else:
                print(f"  - {f.name}/ (directory)")
        if len(all_files) > 20:
            print(f"  ... and {len(all_files) - 20} more files")
else:
    print(f"❌ Output directory not found: {output_path}")


## Troubleshooting

### Nếu thiếu VRAM:
1. Đảm bảo `USE_FP8_DIT = True` và `USE_FP8_VL = True`
2. Đảm bảo `USE_GRADIENT_CHECKPOINTING = True`
3. Nếu vẫn thiếu, set `USE_BLOCKS_TO_SWAP = True` (cần 64GB RAM)
4. Giảm resolution trong `dataset_config.toml` xuống `[960, 544]`

### Nếu gặp lỗi về paths:
- Kiểm tra lại các paths trong cell "Cấu hình Paths"
- Đảm bảo các file models đã được upload đúng vị trí
- Đảm bảo dataset đã được upload vào `dataset/gendata/vietnamese_dataset_qwen_edit/`

### Nếu training bị dừng giữa chừng:
- Checkpoints sẽ được lưu trong `output/`
- Có thể resume training bằng cách thêm `--resume` flag (xem docs)

### Lưu ý quan trọng:
- **Phải dùng `--edit_plus` flag** (không phải `--edit`)
- **Model DiT phải là `qwen_image_edit_2509_bf16.safetensors`** (không phải `qwen_image_edit_bf16.safetensors`)
- Output sẽ được lưu trong thư mục `output/`
